## Идентификация имён


При создании словаря все полные имена читаются из файла, и разбиваются на токены, которые преобразуются во множества.

При обработке сначала строка аналогичным образом разбивается на токены и проверяется, что множество токенов строки является подмножеством токенов имени из словаря. Если это так, то имя добавляется в возвращаемый список. Если в результате перебора словаря имя не нашлось, то запускается проверка на инициалы. Обрабатываются два вида скоращений: Фамилия И.О. и Фамилия И., сокращения могут располагаться как до, так и после фамилии. Если имя не было найдено после проверки на инициалы, то проверяется форма имени, и проверяется ФИО с нормализованным именем.

In [1]:
from common.names_dict import NamesDict
import nest_asyncio
nest_asyncio.apply()

x = NamesDict("resources\\names.xlsx")

print(x.get_names("А.Б. Хармс"))
print(x.get_names("Хармс А.Б."))
print(x.get_names("Хармс АБ"))
print(x.get_names("Витязев АВ"))
print(x.get_names("Белая Г."))
print(x.get_names("Е. Ахматова"))
print(x.get_names("Ахматова"))
print(x.get_names("Ахматова Екатерина Вадимовна"))
print()
print(x.get_names("Александер Хлебников"))
print(x.get_names("Манн Юля"))
print(x.get_names("Московская Женя"))


['Хармс Александр Борисович']
['Хармс Александр Борисович']
['Хармс Александр Борисович']
['Витязев Андрей Викторович']
['Белая Галина']
['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
['Ахматова Екатерина Вадимовна']

['Хлебников Александр']
['Юлия Манн']
['Московская Евгения']


## Работа со сборщиком

Собираем информацию из таблички, в результате обработки выводится пара чисел (необработанные имена, все имена) 

In [1]:
from data_collector import DataCollector
import nest_asyncio
nest_asyncio.apply()

data_path = "data.xlsx"
structure_path = "resources\\survey_structure.json"
names_path = "resources\\names.xlsx"

collector = DataCollector(data_path, structure_path, names_path)
collector.collect()

row 117: "9. Укажите вышестоящего руководителя в формате Фамилия Имя (если вы уже оценили его выше, оставьте вопросы 9-14 незаполненными)"
  Ахматова Екатерина: ['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
row 192: "9. Укажите вышестоящего руководителя в формате Фамилия Имя (если вы уже оценили его выше, оставьте вопросы 9-14 незаполненными)"
  Ахматова Екатерина: ['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']


(2, 373)

Создаём отчётную таблицу

In [ ]:
collector.create_report_df().to_excel("outputs\\report.xlsx", index = False)

c:\Users\zalex\AI-analysis-of-free-survey-responses\data_collector.py:125: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(data=[cur_row], columns = columns_names)], ignore_index = True)


Создаём отчёт по отдельному человеку и типу руководства

In [ ]:
collector.get_person_report("Илюхина Мария", "Непосредственный руководитель")

Помощь в решении проблемных вопросов_positive_pct                                                                                                 1.0
Помощь в решении проблемных вопросов_all                                                                                                            3
Эмоциональный настрой от совместной работы_positive_pct                                                                                           1.0
Эмоциональный настрой от совместной работы_all                                                                                                      3
Экспертность непосредственного руководителя_avg                                                                                              9.333333
Экспертность непосредственного руководителя_count                                                                                                   3
Вклад в улучшение и оптимизацию процессов подразделения_positive_pct                                

Общая оценка экспертности по компании

In [13]:
print(collector.get_average_rating(4))
print(collector.get_average_rating([4]))
print(collector.get_average_rating([4,11]))
print(collector.get_average_rating([4,11,20]))

8.45374449339207
{'Экспертность непосредственного руководителя': 8.45374449339207, 'Overall': 8.45374449339207}
{'Экспертность непосредственного руководителя': 8.45374449339207, 'Экспертность вышестоящего руководителя': 8.384615384615385, 'Overall': 8.433962264150944}
{'Экспертность непосредственного руководителя': 8.45374449339207, 'Экспертность вышестоящего руководителя': 8.384615384615385, 'Экспертность функционального руководителя': 7.5, 'Overall': 8.338983050847459}


Области роста для разных типов руководства

In [15]:
res = collector.get_areas_of_growth()
res["Непосредственный руководитель"]

,question,rating,votes
0,Способствование развитию команды,0.775330,227
1,Процесс предоставления обратной связи,0.801762,227
2,Вклад в улучшение и оптимизацию процессов подр...,0.850220,227
3,Эмоциональный настрой от совместной работы,0.863436,227
4,Помощь в решении проблемных вопросов,0.872247,227


In [16]:
res["Вышестоящий руководитель"]

,question,rating,votes
0,Предоставление информации по ключевым изменени...,0.657143,105
1,Эмоциональный вклад,0.714286,105
2,Вклад в улучшение и оптимизацию процессов подр...,0.761905,105


In [17]:
res["Функциональный руководитель"]

,question,rating,votes
0,Предоставление информации по ключевым изминени...,0.717949,39
1,Объединение кроссфункциональной команды,0.794872,39
2,Эмоциональный настрой в команде,0.846154,39


In [ ]:
collector.get_top_one_level(threshold = 1, top = 100)

,name,rating
0,Шершнев Павел,10.000000
1,Грязнева Екатерина,10.000000
2,Блажник Алексей Евгеньевич,10.000000
3,Критник Евгений,10.000000
4,Московская Евгения,10.000000
...,...,...
56,Инокеньтев Иван,4.571429
57,Тэффи Анастасия,4.000000
58,Комаров Евгений,3.000000
59,Гумилева Нина Александровна,2.000000


In [18]:
res = collector.get_top_several_levels(threshold = 2, top = 10)
res["direct"]

,name,rating direct
0,Гарсиа Юрий,10.000000
1,Маршак Сергей Владимирович,9.800000
2,Губин Иван,9.750000
3,Гумилева Екатерина Николаевна,9.333333
4,Брагин Владимирович Дмитрий,9.000000
5,Хармс Александр Борисович,9.000000
6,Хлебников Александр,9.000000
7,Суворов Денис,8.666667
8,Маяковский Виталий Владимирович,8.333333
9,Ветров Илья Сергеевич,8.200000


In [19]:
res["non direct"]

,name,rating non direct
0,Ведищева Амелия,10.000000
1,Брагин Владимирович Дмитрий,10.000000
2,Хлебников Александр,9.875000
3,Губин Иван,9.666667
4,Гарсиа Юрий,9.666667
5,Гумилева Екатерина Николаевна,9.400000
6,Хармс Александр Борисович,9.400000
7,Ветров Илья Сергеевич,9.166667
8,Юлия Манн,8.090909
9,Суворов Денис,8.000000


In [1]:
from slides_collector import SlidesCollector
import nest_asyncio
nest_asyncio.apply()

data_path = "data.xlsx"
structure_path = "resources\\survey_structure.json"
names_path = "resources\\names.xlsx"
pptx_template_path = "resources\\template.pptx"

collector = SlidesCollector(data_path, structure_path, names_path)
collector.collect()
collector.create_slides("Илюхина", pptx_template_path)
collector.create_slides("Пушкин", pptx_template_path)
collector.create_slides("Витязев", pptx_template_path)
collector.create_slides("диккенс", pptx_template_path)
collector.create_slides("Губин Павел", pptx_template_path)
collector.create_slides("Джон Доу", pptx_template_path)


row 117: "9. Укажите вышестоящего руководителя в формате Фамилия Имя (если вы уже оценили его выше, оставьте вопросы 9-14 незаполненными)"
  Ахматова Екатерина: ['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
row 192: "9. Укажите вышестоящего руководителя в формате Фамилия Имя (если вы уже оценили его выше, оставьте вопросы 9-14 незаполненными)"
  Ахматова Екатерина: ['Ахматова Екатерина Александровна', 'Ахматова Екатерина Вадимовна']
Name "Джон Доу" not found


In [11]:
collector.create_plot("Губин Павел","Непосредственный руководитель")

In [12]:
collector.create_plot("Пушкин Иванович Валерий", "Вышестоящий руководитель")

In [14]:
collector.create_plot("Альфонс Александр", "Функциональный руководитель")